# Refactor the Taxi Fare Prediction Problem with a Pipeline

We will refactor the model you built for the Taxi Fare Prediction Problem using:
- Custom encoders for the distance and time features
- OneHot Encoder in order to encode the hour and day of week features
- SimpleImputer to fill missing values
- A simple linear regression
- A pipeline to put all together

Then: 
- train this pipeline
- apply the pipeline on test data
- generate predictions and submit these new predictions to Kaggle

## First pipeline

In [ ]:
# import the train dataset ( 1000 first rows)
import pandas as pd 
df= pd.read_csv("../data_new_york/train.csv", nrows=1000)
df.head()

In [ ]:
# prepare X and y
X = df.drop(columns="fare_amount")
y = df['fare_amount']

In [ ]:
df.columns

In [ ]:
# Hold out ( train and test dplit )
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train.head()

### Custom transformers

With the Taxi Fare Prediction Challenge data, using `BaseEstimator` and `TransformerMixin`, implement:

- a transformer that computes the haversine distance between the pickup and dropoff locations
- a custom encoder that extracts the time features from `pickup_datetime`

In [ ]:
import numpy as np

def haversine_vectorized(df, 
                         start_lat="pickup_latitude",
                         start_lon="pickup_longitude",
                         end_lat="dropoff_latitude",
                         end_lon="dropoff_longitude"):
    """ 
        Calculates the great circle distance between two points 
        on the earth (specified in decimal degrees).
        Vectorized version of the haversine distance for pandas df.
        Computes the distance in kms.
    """

    lat_1_rad, lon_1_rad = np.radians(df[start_lat].astype(float)), np.radians(df[start_lon].astype(float))
    lat_2_rad, lon_2_rad = np.radians(df[end_lat].astype(float)), np.radians(df[end_lon].astype(float))
    dlon = lon_2_rad - lon_1_rad
    dlat = lat_2_rad - lat_1_rad

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat_1_rad) * np.cos(lat_2_rad) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    return 6371 * c

In [ ]:
# create a DistanceTransformer
from sklearn.base import BaseEstimator, TransformerMixin

class DistanceTransformer(TransformerMixin, BaseEstimator):
    """
        Computes the haversine distance between two GPS points.
        Returns a copy of the DataFrame X with only one column: 'distance'.
        CORRECTION: returns the whole dataframe (impossible to use both pipelines
        at once otherwise)
    """

    def __init__(self,
                 start_lat="pickup_latitude",
                 start_lon="pickup_longitude",
                 end_lat="dropoff_latitude",
                 end_lon="dropoff_longitude"):
        self.start_lat = start_lat
        self.start_lon = start_lon
        self.end_lat = end_lat
        self.end_lon = end_lon
        
    def fit(self, X, y=None): return self

    def transform(self, X, y=None):
    
        X_ = X.copy()
        distance = haversine_vectorized(X)
        X_["distance"] = distance
        return X_[["distance"]]


In [ ]:
# test the DistanceTransformer

dist_trans = DistanceTransformer()
distance = dist_trans.fit_transform(X_train, y_train)
distance.head()

In [ ]:
# create a TimeFeaturesEncoder
class TimeFeaturesEncoder(BaseEstimator, TransformerMixin):
    """
        Extracts the day of week (dow), the hour, the month and the year from a time column.
        Returns a copy of the DataFrame X with only four columns: 'dow', 'hour', 'month', 'year'.
    """

    def __init__(self,datetime="pickup_datetime"):
           self.datetime = datetime

    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        # A COMPLETER 
        X_ = X.copy()
        X_[self.datetime] = pd.to_datetime(X[self.datetime], format='%Y-%m-%d %H:%M:%S UTC')
    
        # extract hour, day of week, month, and year
        X_['hour'] = X_[self.datetime].dt.hour
        X_['dow'] = X_[self.datetime].dt.dayofweek
        X_['month'] = X_[self.datetime].dt.month
        X_['year'] = X_[self.datetime].dt.year
        
        return X_[['dow', 'hour', 'month', 'year']]

In [ ]:
# test the TimeFeaturesEncoder

time_enc = TimeFeaturesEncoder()
time_features = time_enc.fit_transform(X_train, y_train)
time_features.head()

###  Prepocessing pipeline

In [ ]:
# visualizing pipelines in HTML
from sklearn import set_config; set_config(display='diagram')

#### Distance pipeline

Create a pipeline for distances:
- convert the pickup and dropoff coordinates into distances with the DistanceTransformer
- standardize these distances

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
# create distance pipeline dist_pipe

dist_pipe = Pipeline([
    ('dist_trans', DistanceTransformer()),
    ('stdscaler', StandardScaler())
])

# display distance pipeline
dist_pipe

#### Time features pipeline

Create a pipeline for time features
- extract time features from pickup datetime with the TimeFeaturesEncoder
- encode these categorical time features with the OneHotEncoder

In [ ]:
from sklearn.preprocessing import OneHotEncoder
# create time pipeline time_pipe
time_pipe= Pipeline([
    ('time_enc',TimeFeaturesEncoder('pickup_datetime')),
    ('ohe', OneHotEncoder(handle_unknown='ignore'))
])

# display time pipeline
time_pipe

#### Preprocessing pipeline

Wrap up the distance pipeline and the time pipeline into a preprocessing pipeline.

In [ ]:

# create preprocessing pipeline preproc_pipe
from sklearn.pipeline import FeatureUnion

preproc_pipe = FeatureUnion([
    ('distance_pipeline', dist_pipe),
    ('time_features_pipeline', time_pipe)
])
# display preprocessing pipeline
preproc_pipe

### Model pipeline

Create a pipeline containing the preprocessing and the regression model of your choice.

In [ ]:
from sklearn.linear_model import LinearRegression

# Add the model of your choice to the pipeline nammed pipe
pipe = Pipeline([
    ('preprocessing', preproc_pipe),
    ('linear', LinearRegression())
])
# display the pipeline with model
pipe

### Training and performance

Train the pipelined model and compute the prediction on the test set:

In [ ]:
# train the pipelined model
pipe.fit(X_train, y_train)

# compute y_pred on the test set
y_pred = pipe.predict(X_test)

Use the RMSE to evaluate the performance of the model:

In [ ]:
def compute_rmse(y_pred, y_true):
    rmse = np.sqrt(np.mean((y_pred - y_true)**2))
    return rmse

In [ ]:
# call compute_rmse
compute_rmse(y_pred, y_test)

## Complete the workflow with a pipeline

Here we will implement the whole workflow for our Taxifare kaggle challenge.

For that we will refactor the code in functions for more clarity.

Implement the following functions:
- `get_data()` to fetch the data 
- `clean_data()` to clean the data
- `get_pipeline()` to get the pipeline defined earlier
- `train()` to train our model
- `evaluate()` to evaluate our model on test data

In [ ]:
# implement get_data() function
def get_data(path = "../data_new_york/train.csv", nrows=10000):
    '''returns a DataFrame with nrows from s3 bucket'''
    # A COMPLETER
    df = pd.read_csv(path, nrows=nrows)
    return df

In [ ]:
# implement clean_data() function
def clean_data(df, test=False):
    '''returns a DataFrame without outliers and missing values'''
    # A COMPLETER
    df = df[df.fare_amount > 0]
    df = df[df.distance < 100]
    df = df[df.passenger_count <= 8]
    df = df[df.passenger_count > 0]
    
    return df

In [ ]:
# implement set_pipeline() function
def set_pipeline():
    '''returns a pipelined model'''
    pipe = Pipeline([
        ('preprocessing', preproc_pipe),
        ('linear', LinearRegression())
    ])

    return pipe

In [ ]:
# implement train() function
def train(X_train, y_train, pipeline):
    '''returns a trained pipelined model'''
   # A COMPLETER
    pipeline.fit(X_train, y_train)
    return pipeline

In [ ]:
# implement evaluate() function
def evaluate(X_test, y_test, pipeline):
    '''returns the value of the RMSE'''
    # A COMPLETER
    y_pred = pipe.predict(X_test)
    rmse = compute_rmse(y_pred, y_test)
    print(rmse)
    return rmse

In [ ]:
X

### Test the complete worflow

Use the above functions to test the complete workflow.

In [ ]:
# store the data in a DataFrame
df = get_data()

# set X and y
y = df["fare_amount"]
X = df.drop(columns=["fare_amount","key"])

# hold out
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.15)

# build pipeline
pipeline = set_pipeline()

# train the pipeline
train(X_train, y_train, pipeline)

# evaluate the pipeline
rmse = evaluate(X_val, y_val, pipeline)

### Congrats!

Now we are ready to convert this complete workflow into a packaged code 🚀